# Post-hoc  Calibration

This notebook shows how to add conformal calibration when a base detector has already been trained.
It is useful for post-hoc deployment updates where retraining the original detector is not desired.

## Import

This section loads all dependencies used throughout the notebook.

In [1]:
import logging

import numpy as np
import pandas as pd
from oddball import Dataset, load
from pyod.models.hbos import HBOS
from scipy.stats import false_discovery_control

from nonconform import ConformalDetector, Split
from nonconform.metrics import false_discovery_rate, statistical_power

root_logger = logging.getLogger("nonconform")
if not root_logger.handlers:
    root_logger.addHandler(logging.NullHandler())
root_logger.setLevel(logging.ERROR)

## Setup

We partition the training data into a fit subset and a dedicated calibration subset.
Detached calibration means only the calibration split is new to conformalization; the base model is already fit.

In [2]:
x_train, x_test, y_test = load(Dataset.SHUTTLE, setup=True, seed=1)

split_idx = int(0.8 * len(x_train))
x_fit = x_train[:split_idx]
x_calib = x_train[split_idx:]

alpha = 0.1

print(f"x_fit: {x_fit.shape}, x_calib: {x_calib.shape}, x_test: {x_test.shape}")
print(f"Test positives: {int(y_test.sum())}, alpha={alpha}")

x_fit: (18234, 9), x_calib: (4559, 9), x_test: (1000, 9)
Test positives: 100, alpha=0.1


## Post-hoc Calibration Workflow

We first fit HBOS on the fit split, then attach it to `ConformalDetector` and then call `calibrate(...)`.
This is the core pattern for post-training calibration settings.

In [3]:
base_detector = HBOS()
base_detector.fit(x_fit)

ce = ConformalDetector(
    detector=base_detector,
    strategy=Split(n_calib=0.2),
    score_polarity="higher_is_anomalous",
    seed=1,
)

ce.calibrate(x_calib)
p_values = ce.compute_p_values(x_test)

finite_ok = bool(np.isfinite(p_values).all())
range_ok = bool(np.all((0 <= p_values) & (p_values <= 1)))

print(f"Finite p-values: {finite_ok}")
print(f"P-values in [0, 1]: {range_ok}")

Finite p-values: True
P-values in [0, 1]: True


## Evaluation

Finally, we apply BH to p-values and summarize FDR/power to verify end-to-end behavior.

In [4]:
decisions = false_discovery_control(p_values, method="bh") <= alpha

summary = pd.DataFrame(
    [
        {
            "method": "Detached Calibration + BH",
            "discoveries": int(decisions.sum()),
            "fdr": float(false_discovery_rate(y=y_test, y_hat=decisions)),
            "power": float(statistical_power(y=y_test, y_hat=decisions)),
        }
    ]
)

print(summary.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

                   method  discoveries    fdr  power
Detached Calibration + BH          102 0.0784 0.9400
